In [1]:
import sys
sys.path.append('..')

from common.utility import show_implementation

# Fuzzy Inference
Suppose that we have the following inference.

$$\text{IF ROOM IS HOT THEN SPEED OF FAN SHOULD BE HIGH}$$

For crisp inferences, whether the resultant fan speed should be high is easily know.
However, for fuzzy inferences, given a room that is *SOMEWHAT HOT*, what should the fan speed be?

Notice that the $\text{IF-THEN}$ structure is very similar to a [fuzzy relationship](./fuzzy-relation.ipynb#fuzzy-relation).
This is apparent when we consider view the fuzzy relation as 
$$\text{IF THE TEMPERATURE IS HIGH THEN THE PRESSURE IS HIGH}$$
Thus, fuzzy inference is actually just a fuzzy relationship.

## Types of Fuzzy Implication
Suppose that we have the fuzzy set of the antecedent and the consequent, how should be define the membership grade of the relationship?
There are many possible ways to define the above property, as per below:
* Mamdani Implication
    * $\delta(a,b) = \min (a, b)$
* Keleene-Dienes Implication
    * $\delta(a,b) = \max (1 - a, b)$
* Lukasiewicz Implication
    * $\delta(a,b) = \min (1, 1-a+b)$
* Zadeh Implication
    * $\delta(a,b) = \max (\min(a, b), 1-a)$

For this topic, we will be focusing on the **Mamdani Implication**.
In Mamdani Implication, we truncate the consequent by the degree of fulfillment of the antecedent.

Hence, suppose that the fuzzy set of our antecedent is 
$$
0.5/1 + 1/2 + 0.7/3
$$
And the fuzzy set of our consequent is 
$$
0.2/4 + 0.6/5 + 1/6
$$

Then using **Mamdani Implication**, our relationship table would be

| $$\chi_R(x_1, x_2)$$|  4  |  5  |  6  |
| -------             | --- | --- | --- |
| **1**               | 0.2 | 0.5 | 0.5 |
| **2**               | 0.2 | 0.6 | 1   | 
| **3**               | 0.2 | 0.6 | 0.7 |


In [3]:
from module.fuzzy_relation import DiscreteFuzzyRelation

show_implementation(DiscreteFuzzyRelation, 'Section 2')

class DiscreteFuzzyRelation:

    """Section 2: From Inferences"""
    @classmethod
    def from_mamdani(cls, antecedent, consequent):
        table = [[min(x, y) for x in consequent.grades] for y in antecedent.grades]
        return cls(antecedent.values, consequent.values, table)


In [2]:
from module.fuzzy_set import DiscreteFuzzySet

A = DiscreteFuzzySet.from_string('0.5/1+1/2+0.7/3')
B = DiscreteFuzzySet.from_string('0.2/4+0.6/5+1/6')
R = DiscreteFuzzyRelation.from_mamdani(A, B)
print(R)

               4              5              6              
1              0.2            0.5            0.5            
2              0.2            0.6            1.0            
3              0.2            0.6            0.7            


## Determining the Degree of Fulfillment
Suppose our inference is in the form of 
$$
\text{IF A THEN B}
$$

If our fact $A'$ matches $A$ perfectly, then clearly the degree of fulfillment should be perfect, $1$

And if our fact $A'$ does not match $A$ at all, then clearly the degree of fulfillment should be $0$

Now suppose that $A'$ only partially matches $A$, how should be determine the degree of fulfillment?

It is natural to use the area of overlap between the antecedent and $A'$ to determine the degree of fulfillment, since a high overlap means a high degree of matching between $A$ and $A'$.

However, this approach breaks down when we consider $A'$ which perfectly fulfills the antecedent, but has a very small area, ie $A'$ is in the core of $A$ but very thin.
Hence, a better measure for degree of fulfillment is actually the height of the intersection.
Formally, this means that the degree of fulfillment $r$ of an antecedent is defined as 
$$
r(A') = h(A' \cap A)
$$

## Firing Strength
The firing strength $s$ of a rule defines how strongly the rule is fired, in the range of $[0, 1]$.

If there is only 1 antecedent in the inference rule, then $s = r(A')$ and 
$$
B'(y) = \min(r(A'), B(y))
$$

If there are two antecedents connected by an $AND$ relationship, then the firing strength is 
$$
s = \min(r(X'), r(Y'))
$$
where $X, Y$ are the antecedents.

Similarly, if there are two antecedents connected by an $OR$ relationship, then the firing strength is 
$$
s = \max(r(X'), r(Y'))
$$


Now that we have the firing strength, we can determine the conclusion
$$
B'(y) = \min(s, B(y))
$$

## Determining Output of Inference
Suppose that we only have one inference rule, then the output is simply the singular fuzzy function from the consequent after applying the fuzzy inference.

If there are multiple fuzzy rules, then our output is the union of all the consequents.

## Defuzzification
Since most systems require a crisp input, to use the fuzzy output we obtain from our system, we need to transform it into a crisp value.

There are many defuzzification methods, such as
* Center of Area
* Center of Sums

For this topic, we will utilise the center of area method.

### Center of Area
For continuous variable, the center of area is calculated by
$$
x^* = \frac{\int \mu_a(x) x dx}{\int \mu_A(x) dx}
$$
This correspond to the the $x$ value that will split the into two parts of equal area.

For discrete variable, the center of area is calculated by
$$
u^* = \frac{\sum u_i \mu_A(u_i)}{\sum \mu_A(u_i)}
$$